In [2]:
import sympy as sym
import numpy as np
from sympy import symbols, Eq
from sympy.abc import beta, theta, sigma, rho, phi, mu, nu

**Define simbolic variables**

In [3]:
lam_p, lam_w, eps_p, eps_w,phi_pi=symbols('\lambda_p , \lambda_w ,\epsilon_p,\epsilon_w,\phi_{\pi}')
p,w,y,i,pi_p,pi_w = symbols('p_t,w_t,y_t,i_t,\pi^{p}_t,\pi^{w}_t')
p1,w1,y1,i1,pi_p1,pi_w1 = symbols('p_{t+1},w_{t+1},y_{t+1},i_{t+1},\pi^{p}_{t+1},\pi^{w}_{t+1}')
pt,wt,pit_p, pit_w=symbols('\\tilde{p}_t,\\tilde{w}_t,\\tilde{\pi}^{p}_t,\\tilde{\pi}^{w}_t')
pt1,wt1,pit_p1, pit_w1=symbols('\\tilde{p}_{t+1},\\tilde{w}_{t+1},\\tilde{\pi}^{p}_{t+1},\\tilde{\pi}^{w}_{t+1}')
mu = sym.log(eps_p/(eps_p-1))

**Define Equilibrium Equations**

In [4]:
eq1  = Eq(pi_p1,p1-p)
eq2  = Eq(pi_w1,w1-w)
eq3  = Eq(pit_p1,pt1-pt)
eq4  = Eq(pit_w1,wt1-wt)
eq5  = Eq((phi+sigma)*y1,w1-p1)
eq6  = Eq((1-theta)*eps_p*(p1-pt1),theta*eps_w*(w1-wt1))
eq7  = Eq(y, y1-1/sigma*(rho+phi_pi*pi_p1+nu-pi_p1-rho))
eq8  = Eq(pit_p, (1-lam_p)*beta*pit_p1+(1-beta*(1-lam_p))*(p-pt-eps_w/(eps_w+eps_p)*(p-w-mu)))
eq9  = Eq(pit_w, (1-lam_w)*beta*pit_w1+(1-beta*(1-lam_w))*(w-wt+eps_p/(eps_w+eps_p)*(p-w-mu)))

$$\Gamma_0 y_t =\Gamma_1 y_{t-1}+C+\Psi z_t+\Pi\eta_t$$

$y_t = (E_t\{p_{t+1}\},E_t\{\tilde p_{t+1}\}, E_t\{\pi_{t+1}^p\},E_t\{\tilde\pi_{t+1}^p\}, E_t\{w_{t+1}\}, E_t\{\tilde w_{t+1}\}, E_t\{\pi_{t+1}^w\}, E_t\{\tilde\pi_{t+1}^w\},E_t\{y_{t+1}\})$

In [30]:
x = [p1,w1,pt1,wt1,y1,pi_p1,pi_w1,pit_p1,pit_w1]
x_ =[p, w, pt, wt, y, pi_p, pi_w, pit_p, pit_w]
z = [nu]

**Define Matrices**

In [31]:
Gamma0,_ = sym.linear_eq_to_matrix([eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9],x)
Gamma1,_ = sym.linear_eq_to_matrix(_,x_)
Psi, C = sym.linear_eq_to_matrix(_, z)
Eta = sym.Matrix([[1,0,0,0,0,0,0,0,0],
                  [0,1,0,0,0,0,0,0,0]]).T

**Calibrate**

In [32]:
calibration={'beta':0.98, 'theta':1/2,'sigma':2,'phi':2,'\epsilon_w':5,'\epsilon_p':5,
'\lambda_p':0.25, '\lambda_w':0.25, '\phi_{\pi}':1.2}

In [33]:
Gamma0 = Gamma0.subs(calibration)
Gamma1 = Gamma1.subs(calibration)
C = C.subs(calibration)
Psi = Psi.subs(calibration)
Eta = Eta.subs(calibration)

In [34]:
Gamma0

Matrix([
[ -1,    0,    0,   0,  0,   1, 0,      0,      0],
[  0,   -1,    0,   0,  0,   0, 1,      0,      0],
[  0,    0,   -1,   0,  0,   0, 0,      1,      0],
[  0,    0,    0,  -1,  0,   0, 0,      0,      1],
[  1,   -1,    0,   0,  4,   0, 0,      0,      0],
[2.5, -2.5, -2.5, 2.5,  0,   0, 0,      0,      0],
[  0,    0,    0,   0, -1, 0.1, 0,      0,      0],
[  0,    0,    0,   0,  0,   0, 0, -0.735,      0],
[  0,    0,    0,   0,  0,   0, 0,      0, -0.735]])

In [ ]:
Gamma0_inv=Gamma0.inv()
Gamma1, C, Psi, Eta = Gamma0_inv@Gamma1, Gamma0_inv@C, Gamma0_inv@Psi, Gamma0_inv@Eta 
P, Lambda = Gamma1.diagonalize()
P_inv = P.inv()
ind = list(np.argsort(np.abs(np.diag(np.abs(Lambda)))))
Lambda = sym.diag(*[Lambda[i,i] for i in ind])
P=P.permute_cols(ind)

In [ ]:
n_s = len([Lambda[i,i] for i in range(Lambda.rows) if np.abs(Lambda[i,i])<1])

In [13]:
def solve_system(Gamma0:sym.Matrix, Gamma1:sym.Matrix, C:sym.Matrix, Psi:sym.Matrix, Eta:sym.Matrix)->tuple:
    if np.abs(Gamma0.det())<1e-10:
        raise ValueError('Gamma0 is not invertible')
    Gamma0_inv=Gamma0.inv()
    Gamma1, C, Psi, Eta = Gamma0_inv@Gamma1, Gamma0_inv@C, Gamma0_inv@Psi, Gamma0_inv@Eta 
    P, Lambda = Gamma1.diagonalize()
    P_inv = P.inv()
    ind = list(np.argsort(np.abs(np.diag(np.abs(Lambda)))))
    Lambda = sym.diag(*[Lambda[i,i] for i in ind])
    P=P.permute_cols(ind)
    n_s = len([Lambda[i,i] for i in range(Lambda.rows) if np.abs(Lambda[i,i])<1])
    n_u = Lambda.rows-n_s
    Lambda_S=Lambda[:n_s,:n_s]
    Lambda_U=Lambda[n_s:,n_s:]
    Theta_1 = P*sym.diag(Lambda_S, sym.zeros((n_u,n_u)))*P_inv
    #Theta_c = P*sym.Matrix([[sym.eye(n_s)],(sym.eye(n_u)-Lambda_U).inv()])*P_inv
    #Theta_z = P*sym.Matrix([[sym.eye(Psi.rows), Psi],[0,0]])*P_inv
    Theta_c = None
    Theta_z = None
    return Theta_1, Theta_c, Theta_z

In [ ]:
Theta_1, Theta_c, Theta_z = solve_system(Gamma0, Gamma1, C, Psi, Eta)

## Galí (2015) Sticky wages and sticky prices

In [32]:
import sympy as sym
import numpy as np
from sympy import symbols, Eq
from sympy.abc import alpha, beta, theta, sigma, rho, phi, mu, nu
y,pi_p, pi_w, w_ = symbols('y_t,\pi^p_t,\pi_t^w,\omega_{t-1}')
y1,pi_p1, pi_w1, w = symbols('E_ty_{t+1},E_t\pi^p_{t+1},E_t\pi_{t+1}^w,\omega_t')

phi_y, phi_p, phi_w=symbols('\phi_y, \phi_p, \phi_w')
lam_p, lam_w = symbols('\lambda_p, \lambda_w')
chi_p, chi_w = symbols('chi_p, chi_w')

v, Delta_w = symbols('v, \Delta\omega_t^n')

In [33]:
i = rho+phi_p*pi_p+phi_w*pi_w+phi_y*y+v

In [34]:
is_curve = Eq(y, y1-1/sigma*(i-pi_p1-rho))
real_wage = Eq(w, w_+pi_w-pi_p-Delta_w)
price_nkpc = Eq(pi_w, beta*pi_w1+chi_w*y-lam_w*w)
wage_nkpc = Eq(pi_p, beta*pi_p1+chi_p*y+lam_p*w)

In [35]:
var  = y1,pi_p1, pi_w1, w
var_ = y,pi_p, pi_w, w_
z = v, Delta_w

In [36]:
Gamma0,_ = sym.linear_eq_to_matrix([is_curve,real_wage,price_nkpc,wage_nkpc], var)
Gamma1,_ = sym.linear_eq_to_matrix(_,var_)
Psi, C = sym.linear_eq_to_matrix(_, z)
Eta = sym.Matrix([[0,1,0,0]]).T

In [37]:
calibration={'beta':0.99, 'phi':5, 'alpha':1/4, 'eps_p':9,
             '\phi_p':1.5,'\phi_w':0.125, '\phi_y':0.5/4,
             'eps_w':4.5, 'sigma':1,
              'theta_p':3/4,'theta_w':3/4}

In [38]:
class StickyWageNK():
    def __init__(self):
        self.calibration = None
    def calibrate(self, calibration: dict):
        theta_w = calibration['theta_w']
        theta_p = calibration['theta_p']
        beta = calibration['beta']
        phi = calibration['phi']
        eps_w = calibration['eps_w']
        eps_p = calibration['eps_w']
        alpha = calibration['alpha']
        sigma = calibration['sigma']
        calibration['\lambda_w']=(1-theta_w)*(1-beta*theta_w)/(theta_w*(1+eps_w*phi))
        calibration['\lambda_p']=(1-theta_p)*(1-beta*theta_p)/(theta_p)*(1-alpha)/(1-alpha+alpha*eps_p)
        calibration['chi_p'] = alpha*calibration['\lambda_p']/(1-alpha)
        calibration['chi_w'] = calibration['\lambda_w']*(sigma+phi/(1-alpha))
        self.calibration = calibration

In [39]:
economy=StickyWageNK()
economy.calibrate(calibration)

In [40]:
calibration = economy.calibration

In [41]:
Gamma0 = Gamma0.subs(calibration)
Gamma1 = Gamma1.subs(calibration)
C = C.subs(calibration)
Psi = Psi.subs(calibration)

In [ ]:
Gamma0_inv=Gamma0.inv()
Gamma1, C, Psi, Eta = Gamma0_inv@Gamma1, Gamma0_inv@C, Gamma0_inv@Psi, Gamma0_inv@Eta 
P, Lambda = Gamma1.diagonalize()
ind = list(np.argsort(np.abs(np.diag(np.abs(Lambda)))))
Lambda = sym.diag(*[Lambda[i,i] for i in ind])
P=P.permute_cols(ind)
P_inv = P.inv()
n_s = len([Lambda[i,i] for i in range(Lambda.rows) if np.abs(Lambda[i,i])<1])
n_u = Lambda.rows-n_s
Lambda_S=Lambda[:n_s,:n_s]
Lambda_U=Lambda[n_s:,n_s:]
Theta_1 = P[:,:n_s]*Lambda_S*P_inv[:n_s,:]
Theta_c = P[:,:n_s]*P_inv[:n_s,:]+P[:,n_s:]*(sym.eye(n_u)-Lambda_U)*P_inv[n_s:,:]
Phi = P[:,:n_s]*Eta*(P_inv[n_s:,:]*Eta).inv()
Theta_z = P[:,:n_s]*P_inv[:n_s,:]-P[:,:n_s]*Phi*P_inv[n_s:,:]

In [30]:
P_inv[n_s:,:]

Matrix([
[-0.572221626323231 + 0.0886585437531521*I, -0.423030301569504 - 0.651789725522227*I,  -1.46119245032438 - 1.62437436460687*I, -0.176201644740351 + 0.0201962910582168*I],
[ -0.0742595078742411 - 0.51302046788616*I, -0.668855428050685 - 0.191048128426249*I, -1.77367582333268 - 0.824409383235328*I, -0.0289465770003362 - 0.156108143050026*I],
[-0.245537947552287 - 0.0752726868623043*I,  -2.58608137004319 - 0.792795146771546*I,  1.91647411004567 + 0.587518780714989*I,   0.374572120934527 + 0.114829704522351*I]])

In [31]:
Eta

Matrix([
[ 0.0346801346801347],
[-0.0346801346801347],
[0.00368937602980156],
[                  1]])

In [201]:
(sym.eye(n_u)-Lambda_U).inv()

Matrix([
[-4.52593640537801 - 6.52170473917186*I,                                      0,                                          0],
[                                     0, -4.52593640537801 + 6.52170473917186*I,                                          0],
[                                     0,                                      0, -4.24487837822232 - 7.91340106096718e-64*I]])

In [ ]:
def solve_system(Gamma0:sym.Matrix, Gamma1:sym.Matrix, C:sym.Matrix, Psi:sym.Matrix, Eta:sym.Matrix)->tuple:
    if np.abs(Gamma0.det())<1e-10:
        raise ValueError('Gamma0 is not invertible')
    Gamma0_inv=Gamma0.inv()
    Gamma_1, C, Psi, Eta = Gamma0_inv@Gamma_1, Gamma0_inv@C, Gamma0_inv@Psi, Gamma0_inv@Eta 
    P, Lambda = Gamma1.diagonalize()
    P_inv = P.inv()
    ind = list(np.argsort(np.abs(np.diag(np.abs(Lambda)))))
    Lambda = sym.diag(*[Lambda[i,i] for i in ind])
    P=P.permute_cols(ind)
    n_s = len([Lambda[i,i] for i in range(Lambda.rows) if np.abs(Lambda[i,i])<1])
    n_u = Lambda.rows-n_s
    Lambda_S=Lambda[:n_s,:n_s]
    Lambda_U=Lambda[n_s:,n_s:]
    Theta_1 = P*sym.diag(Lambda_S, sym.zeros((n_u,n_u)))*P_inv
    #Theta_c = P*sym.Matrix([[sym.eye(n_s)],(sym.eye(n_u)-Lambda_U).inv()])*P_inv
    #Theta_z = P*sym.Matrix([[sym.eye(Psi.rows), Psi],[0,0]])*P_inv
    Theta_c = None
    Theta_z = None
    return Theta_1, Theta_c, Theta_z